In [8]:
import pandas as pd
import numpy as np
import re
import os

import trmc.load as load
import trmc.analysis as analysis

import xarray as xr

from IPython.display import clear_output
from collections import OrderedDict
import itertools

In [16]:
# basedir = '\\\\depot.engr.oregonstate.edu\\users\\coe_apirate\\Windows.Documents\\Desktop\\Data'
basedir = 'C:\\Users\\aspit\\OneDrive\\Data\\TRMC\\Raw Data'
fps = {
    'bia': os.path.join(basedir,'20190128\Bi_A_2'),
    'bid': os.path.join(basedir,'20190201\Sample_D_2'),
    'bic': os.path.join(basedir,'20190130\Sample Bi_C_2_2'),
    'bib': os.path.join(basedir,'20190202\Sample Bi_B_2'),
#     'bhjua': os.path.join(basedir,'20190125\PH3TUA'),
#     'bhja': os.path.join(basedir,'20190127\P3HTA')   
}

In [17]:
da_sw = load.sweeps2ds(fps)['Vsignal(V)']

## load in and plot the deltaV data sets

In [18]:
# timeshifts = {'bhjua': -270e-9,'bhja': -270e-9, 'bid': -270e-9,'bib': -270e-9}
timefactor = {'bia': 2, 'bic' : 2}

#corect for incorrect data entry, most are right
fluencefactor = {
    'bia': 5.53665E+14/5.53665E+14,
    'bid': 4.21267E+14/4.213E+14,
    'bic': 5.29593E+14/3.611E+14,
    'bib': 3.852E+14/3.852E+14,
}

In [19]:
das = []
das_bv = []

for samp in fps:
    direc = os.path.join(fps[samp],'FreqFluence')
    s_fps = load.freqfluence_flist(direc)
    print('Loading data for ' + samp)
    s,backvs = load.freqfluence_load(s_fps)
    print('Converting to Dataset')
    da = xr.DataArray.from_series(s)
    da = da.chunk({'time': 2000})
    da = da.expand_dims('sample')
    da['sample'] = [samp]
    
    if samp in timefactor:
#         print('shifting time for ' + samp + " by " + str(timeshifts[samp]))
        print('scaling time for ' + samp + " by " + str(timefactor[samp]))
        timearr = da.indexes['time']
        da = da.assign_coords(time =  timearr*timefactor[samp])
        
    if samp in fluencefactor:
        print('Multiplying fluence of ' + samp + " by " + str(fluencefactor[samp]))
        fluencearr = da.indexes['fluence']
        da = da.assign_coords(fluence =  fluencearr*fluencefactor[samp])
    
    das.append(da)
    
    da_bv = xr.DataArray.from_series(backvs)
    da_bv = da_bv.expand_dims('sample')
    da_bv['sample'] = [samp]
    das_bv.append(da_bv)
    
da_dv = xr.concat(das, dim = 'sample')
da_dv = da_dv*(-1) #voltage is flipped
da_bv = xr.concat(das_bv,dim = 'sample')
print('done')

no match for file TestSweep_psm_image.png
Loading data for bib
Converting to Dataset
Multiplying fluence of bib by 1.0
done


In [6]:
das = []
for samp in fps:
    direc = os.path.join(fps[samp],'FreqFluence')    
    fpsdcs = load.freqdcs_flist(direc)
    
    for direction,freqt in fpsdcs.index:
        fp = fpsdcs.loc[direction,freqt]
        s = load.loadsweep(fp,defaultV = 0.025)
        s = s.rename(s.name.replace(' ', ''))
        s.index = s.index.rename('freq')
        da = xr.DataArray.from_series(s)
        da = da.assign_coords(direction = direction).expand_dims('direction')
        da = da.assign_coords(freqt = freqt).expand_dims('freqt')
        da = da.assign_coords(sample = samp).expand_dims('sample')
        das.append(da)
    
da_dcs = xr.merge(das)['Vsignal(V)']

In [22]:
%store da_dv
%store da_bv
%store da_sw
%store da_dcs

Stored 'da_dv' (DataArray)
Stored 'da_bv' (DataArray)
Stored 'da_sw' (DataArray)
Stored 'da_dcs' (DataArray)
